In [1]:
import pandas as pd

In [2]:
df = pd.read_json('relevant_data.json', orient='records')
df.head(5)

,title,obj_short,obj_long,selection_criteria_short,selection_criteria_studies,selection_criteria_p,selection_criteria_i,selection_criteria_o,selection_criteria_long,long_methods_only,study_title,study_abstract,label
0,Mindfulness‐based psychological interventions ...,Objectives To assess the effects of psychologi...,To assess the effects of psychological interve...,Selection criteria We included randomised cont...,"Randomised controlled trials (RCTs), including...",Eligible participants included any student stu...,Experimental intervention We included any psyc...,"Primary outcomes Anxiety, measured using valid...",None,None,Preliminary Efficacy of a Brief Mindfulness In...,Objectives Simulation‐based mastery learning...,included
1,Mindfulness‐based psychological interventions ...,Objectives To assess the effects of psychologi...,To assess the effects of psychological interve...,Selection criteria We included randomised cont...,"Randomised controlled trials (RCTs), including...",Eligible participants included any student stu...,Experimental intervention We included any psyc...,"Primary outcomes Anxiety, measured using valid...",None,None,Effects of a Required Large-Group Mindfulness ...,Background Teaching mindfulness techniques h...,included
2,Mindfulness‐based psychological interventions ...,Objectives To assess the effects of psychologi...,To assess the effects of psychological interve...,Selection criteria We included randomised cont...,"Randomised controlled trials (RCTs), including...",Eligible participants included any student stu...,Experimental intervention We included any psyc...,"Primary outcomes Anxiety, measured using valid...",None,None,Breathing meditation by medical students at Kh...,OBJECTIVE To examine the short‐term effects ...,included
3,Mindfulness‐based psychological interventions ...,Objectives To assess the effects of psychologi...,To assess the effects of psychological interve...,Selection criteria We included randomised cont...,"Randomised controlled trials (RCTs), including...",Eligible participants included any student stu...,Experimental intervention We included any psyc...,"Primary outcomes Anxiety, measured using valid...",None,None,"Mindfulness for medical, surgical and psychiat...",BACKGROUND Mindfulness based stress reductio...,included
4,Mindfulness‐based psychological interventions ...,Objectives To assess the effects of psychologi...,To assess the effects of psychological interve...,Selection criteria We included randomised cont...,"Randomised controlled trials (RCTs), including...",Eligible participants included any student stu...,Experimental intervention We included any psyc...,"Primary outcomes Anxiety, measured using valid...",None,None,Teaching an experiential mind-body method to m...,OBJECTIVE The authors investigate whether in...,excluded


In [3]:
df = df.drop_duplicates()

In [4]:
df_filtered = df.drop(columns=['long_methods_only', 'label'])

In [5]:
df_filtered.notnull().value_counts().sort_values(ascending=False)

title  obj_short  obj_long  selection_criteria_short  selection_criteria_studies  selection_criteria_p  selection_criteria_i  selection_criteria_o  selection_criteria_long  study_title  study_abstract
True   True       True      True                      True                        True                  True                  True                  False                    True         True              88725
                                                                                                        False                 False                 False                    True         True               4827
                                                      False                       False                 False                 False                 False                    True         True                297
                                                      True                        False                 False                 False                 False                

In [6]:
grouped = df_filtered.notnull().groupby([ 'selection_criteria_short', 'study_title', 'study_abstract', 'title', 'obj_short', 'obj_long', 'selection_criteria_long', 'selection_criteria_studies', 'selection_criteria_p', 'selection_criteria_i', 'selection_criteria_o']).apply(lambda x: x.index)

In [7]:
grouped.keys().names

FrozenList(['selection_criteria_short', 'study_title', 'study_abstract', 'title', 'obj_short', 'obj_long', 'selection_criteria_long', 'selection_criteria_studies', 'selection_criteria_p', 'selection_criteria_i', 'selection_criteria_o'])

In [8]:
pd.set_option('display.expand_frame_repr', False)

In [9]:
for k, v in grouped.items():
    print('='*20)
    print(list(zip(grouped.keys().names, k)))
    print(k)
    split = df_filtered.loc[v]
    print(len(split))
    tokenized = split.transform(lambda x: x.str.split().str.len())
    print(tokenized.sum(axis=1).agg(['mean', 'max', 'min']))
    print('-----')
    print(tokenized.agg(['mean', 'max', 'min']))

[('selection_criteria_short', True), ('study_title', True), ('study_abstract', True), ('title', True), ('obj_short', True), ('obj_long', True), ('selection_criteria_long', False), ('selection_criteria_studies', False), ('selection_criteria_p', False), ('selection_criteria_i', False), ('selection_criteria_o', False)]
(True, True, True, True, True, True, False, False, False, False, False)
297
mean    575.478114
max     926.000000
min     231.000000
dtype: float64
-----
          title   obj_short    obj_long  selection_criteria_short  selection_criteria_studies  selection_criteria_p  selection_criteria_i  selection_criteria_o  selection_criteria_long  study_title  study_abstract
mean  11.892256   44.138047  174.087542                 64.218855                         NaN                   NaN                   NaN                   NaN                      NaN    16.676768      264.464646
max   24.000000  110.000000  211.000000                114.000000                         NaN       

In [23]:
from transformers import AutoTokenizer
from pandarallel import pandarallel

pandarallel.initialize()

tokenizer = AutoTokenizer.from_pretrained('elinas/llama-7b-hf-transformers-4.29')

INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [24]:
for k, v in grouped.items():
    print('='*20)
    print(list(zip(grouped.keys().names, k)))
    print(k)
    split = df_filtered.loc[v]
    print(len(split))
    tokenized = split.fillna('').parallel_applymap(lambda x: tokenizer.encode(x, add_special_tokens=False), axis=0).transform(lambda x: x.str.len())
    print(tokenized.sum(axis=1).agg(['mean', 'max', 'min']))
    print('-----')
    print(tokenized.agg(['mean', 'max', 'min']))

[('selection_criteria_short', True), ('study_title', True), ('study_abstract', True), ('title', True), ('obj_short', True), ('obj_long', True), ('selection_criteria_long', False), ('selection_criteria_studies', False), ('selection_criteria_p', False), ('selection_criteria_i', False), ('selection_criteria_o', False)]
(True, True, True, True, True, True, False, False, False, False, False)
297
mean    1073.158249
max     2101.000000
min      424.000000
dtype: float64
-----
          title  obj_short    obj_long  selection_criteria_short  selection_criteria_studies  selection_criteria_p  selection_criteria_i  selection_criteria_o  selection_criteria_long  study_title  study_abstract
mean  20.420875   70.79798  339.925926                100.424242                         0.0                   0.0                   0.0                   0.0                      0.0    31.013468      510.575758
max   44.000000  172.00000  446.000000                183.000000                         0.0       

In [ ]:
df_2 = pd.read_json('test.json', orient='records')
df_2['Title'] = df_2['meta'].apply(lambda x: x['title'])
df_2

,id,meta,abstract,data,references,Title
0,https://doi.org/10.1002/14651858.CD010822.pub4,"{'authors': ['Akshay Sharma', 'Manu Easow Math...",{'background': 'Background Haemophilia is a ge...,{'conclusions': ' Implications for practice...,{'Additional references': [{'id': ' Anson 198...,Gene therapy for haemophilia
1,https://doi.org/10.1002/14651858.CD009326.pub4,"{'authors': ['Naohiro Yonemoto', 'Shuko Nagai'...",{'background': 'Background Maternal complicati...,{'conclusions': ' Implications for practice...,{'References to studies included in this revie...,Schedules for home visits in the early postpar...
2,https://doi.org/10.1002/14651858.CD008288.pub2,"{'authors': ['Kalpana Sridharan', 'Roshni Moha...",{'background': 'Background Patients with diabe...,{'conclusions': ' Implications for practice...,{'References to studies included in this revie...,Ayurvedic treatments for diabetes mellitus
3,https://doi.org/10.1002/14651858.CD004073.pub4,"{'authors': ['Olukunmi O Balogun', 'Katharina ...",{'background': 'Background Miscarriage is a co...,{'conclusions': ' Implications for practice...,{'References to studies included in this revie...,Vitamin supplementation for preventing miscarr...
4,https://doi.org/10.1002/14651858.CD012213.pub2,"{'authors': ['Sumanth Kumbargere Nagraj', 'Pra...",{'background': 'Background Halitosis or bad br...,{'conclusions': ' Implications for practice...,{'References to studies included in this revie...,Interventions for managing halitosis
...,...,...,...,...,...,...
362,https://doi.org/10.1002/14651858.CD005644.pub2,"{'authors': ['Chukwuemeka E Nwachukwu', 'Josep...",{'background': 'Background AIDS‐related diarrh...,{'conclusions': ' Implications for practice...,{'References to studies included in this revie...,Antimotility agents for chronic diarrhoea in p...
363,https://doi.org/10.1002/14651858.CD003264.pub3,"{'authors': ['Elisabeth M Hodson', 'Narelle S ...",{'background': 'Background Growth retardation ...,{'conclusions': ' Implications for practice...,{'References to studies included in this revie...,Growth hormone for children with chronic kidne...
364,https://doi.org/10.1002/14651858.CD007112.pub3,"{'authors': ['George Settas', 'Clare Settas', ...","{'background': 'Background Hyperopia, or hyper...",{'conclusions': ' Implications for practice...,{'References to studies excluded from this rev...,Photorefractive keratectomy (PRK) versus laser...
365,https://doi.org/10.1002/14651858.CD010173.pub3,"{'authors': ['Tanya Walsh', 'Saman Warnakulasu...",{'background': 'Background The early detection...,{'conclusions': ' Implications for practice...,{'References to studies included in this revie...,Clinical assessment for the detection of oral ...


In [ ]:
df_2[df_2['Title'].isin(df['Title'])]

,id,meta,abstract,data,references,Title


In [ ]:
df = pd.read_json('instruct_cochrane_eval_v2_nogold.json')
df

,instruction,input,output
0,"Given the abstract, what is the study's Outcome?",Abstract: Objectives Verify the influence of o...,Outcome: 1. Incidence of VAP 2. Duration of ve...
1,"Given the abstract, what is the study's Compar...",Abstract: One hundred and seventy two 0. 5 3. ...,Comparison: Randomised trial conducted in a ru...
2,"Given the abstract, objectives and selection c...",Abstract: Nutrition interventions have an effe...,Excluded because This study included moderatel...
3,"Given the abstract, what is the study's Interv...",Abstract: BACKGROUND Depression is common in p...,Intervention: Stated purpose to explore the fe...
4,"Given the abstract, what is the study's Interv...",Abstract: In the light of the current controve...,"Intervention: Treatment pentoxifylline, 1200 m..."
...,...,...,...
11920,"Given the abstract, objectives and selection c...",Abstract: One hundred and seven adult outpatie...,Excluded because Open controlled trial
11921,"Given the abstract, objectives and selection c...",Abstract: The efficacy and safety of adding Sa...,Excluded because Study of amoebiasis associate...
11922,"Given the abstract, what is the study's Outcome?",Abstract: The purpose of the present study was...,"Outcome: Primary Suicidality, assessed with th..."
11923,"Given the abstract, what is the study's Compar...",Abstract: Background aims Older people are vul...,Comparison: RCT. Parallel design with 2 treatm...


In [ ]:
inc_exc = df[df['instruction'].str.contains('should the study be included or excluded?')]
inc_exc

,instruction,input,output
2,"Given the abstract, objectives and selection c...",Abstract: Nutrition interventions have an effe...,Excluded because This study included moderatel...
7,"Given the abstract, objectives and selection c...",Abstract: Objective To determine whether the b...,Included
8,"Given the abstract, objectives and selection c...",Abstract: PURPOSE To explore the effect of exp...,Excluded because Not a task shifting intervent...
11,"Given the abstract, objectives and selection c...",Abstract: BACKGROUND The current study was con...,Included
14,"Given the abstract, objectives and selection c...",Abstract: The current study evaluated the effe...,Excluded because Intervention did not contain ...
...,...,...,...
11918,"Given the abstract, objectives and selection c...",Abstract: BACKGROUND Most newborn deaths occur...,Excluded because Women were enrolled in the an...
11919,"Given the abstract, objectives and selection c...",Abstract: Purpose To assess the influence of a...,Excluded because Non RCT.
11920,"Given the abstract, objectives and selection c...",Abstract: One hundred and seven adult outpatie...,Excluded because Open controlled trial
11921,"Given the abstract, objectives and selection c...",Abstract: The efficacy and safety of adding Sa...,Excluded because Study of amoebiasis associate...


In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')

def clean_text(text, stopword_removal=True, lowercase=True):
    text = text.lower() if lowercase else text

    stop_words = set(stopwords.words('english'))
    text = ' '.join([word for word in text.split() if word not in stop_words]) if stopword_removal else text
    
    text = nltk.WordPunctTokenizer().tokenize(text)
    lemm = nltk.stem.WordNetLemmatizer()
    text = list(map(lambda word: lemm.lemmatize(word), text))

    return text

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/william/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/william/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
label_to_idx = {'Included': 1, 'Excluded': 0}
idx_to_label = {idx: label for label, idx in label_to_idx.items()}

inc_exc[['abstract', 'review']] = inc_exc['input'].apply(lambda x: x.split('Objectives: ')).apply(pd.Series)
inc_exc['abstract'] = inc_exc['abstract'].str.replace('Abstract: ', '').apply(clean_text)
inc_exc['label'] = inc_exc['output'].str.split().str[0].apply(lambda x: label_to_idx[x]) # Remove justification for exclude
inc_exc.head(5)

/tmp/ipykernel_188996/2295107791.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inc_exc[['abstract', 'review']] = inc_exc['input'].apply(lambda x: x.split('Objectives: ')).apply(pd.Series)
/tmp/ipykernel_188996/2295107791.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inc_exc[['abstract', 'review']] = inc_exc['input'].apply(lambda x: x.split('Objectives: ')).apply(pd.Series)
/tmp/ipykernel_188996/2295107791.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

,instruction,input,output,abstract,review,label
2,"Given the abstract, objectives and selection c...",Abstract: Nutrition interventions have an effe...,Excluded because This study included moderatel...,"[nutrition, intervention, effect, growth, ,, e...",To assess the effects and safety of preventive...,0
7,"Given the abstract, objectives and selection c...",Abstract: Objective To determine whether the b...,Included,"[objective, determine, whether, benefit, caffe...",To determine the effect of prophylactic treatm...,1
8,"Given the abstract, objectives and selection c...",Abstract: PURPOSE To explore the effect of exp...,Excluded because Not a task shifting intervent...,"[purpose, explore, effect, expressive, writing...",To evaluate the effectiveness of PW led treatm...,0
11,"Given the abstract, objectives and selection c...",Abstract: BACKGROUND The current study was con...,Included,"[background, current, study, conducted, determ...",To assess the efficacy of medical intervention...,1
14,"Given the abstract, objectives and selection c...",Abstract: The current study evaluated the effe...,Excluded because Intervention did not contain ...,"[current, study, evaluated, effectiveness, wid...",To determine whether competitions lead to high...,0


In [ ]:
'exclude' in set(inc_exc['abstract'][:1].values[0])

False

In [ ]:
len(inc_exc.groupby('review'))

409

In [ ]:
filtered_df = inc_exc.groupby('review').filter(lambda x: x['instruction'].count() > 100)
print(filtered_df)
filtered_reviews = filtered_df.groupby(filtered_df['input'].str.split('Objectives:').str[1])
filtered_reviews.count().head(3)

                                             instruction  \
8      Given the abstract, objectives and selection c...   
16     Given the abstract, objectives and selection c...   
19     Given the abstract, objectives and selection c...   
22     Given the abstract, objectives and selection c...   
43     Given the abstract, objectives and selection c...   
...                                                  ...   
11880  Given the abstract, objectives and selection c...   
11897  Given the abstract, objectives and selection c...   
11904  Given the abstract, objectives and selection c...   
11916  Given the abstract, objectives and selection c...   
11924  Given the abstract, objectives and selection c...   

                                                   input  \
8      Abstract: PURPOSE To explore the effect of exp...   
16     Abstract: OBJECTIVE To investigate the feasibi...   
19     Abstract: BACKGROUND The Democratic Republic o...   
22     Abstract: Background Albiglutide

,instruction,input,output,abstract,review,label
input,,,,,,
"The objectives of this review were to assess the effects of various interventions used to control halitosis due to oral diseases only. We excluded studies including patients with halitosis secondary to systemic disease and halitosis masking interventions. \n Selection Criteria: We included randomised controlled trials RCTs which involved adults over the age of 16, and any intervention for managing halitosis compared to another or placebo, or no intervention. The active interventions or controls were administered over a minimum of one week and with no upper time limit. We excluded quasi randomised trials, trials comparing the results for less than one week follow up, and studies including advanced periodontitis.",108,108,108,108,108,108
"The primary aims of the review are to examine the effectiveness of strategies aiming to improve the implementation of school based policies, programs or practices to address child diet, physical activity, obesity, tobacco or alcohol use. Secondary objectives of the review are to Examine the effectiveness of implementation strategies on health behaviour e. g. fruit and vegetable consumption and anthropometric outcomes e. g. BMI, weight; describe the impact of such strategies on the knowledge, skills or attitudes of school staff involved in implementing health promoting policies, programs or practices; describe the cost or cost effectiveness of such strategies; and describe any unintended adverse effects of strategies on schools, school staff or children. \n Selection Criteria: 'Implementation' was defined as the use of strategies to adopt and integrate evidence based health interventions and to change practice patterns within specific settings. We included any trial randomised or non randomised conducted at any scale, with a parallel control group that compared a strategy to implement policies or practices to address diet, physical activity, overweight or obesity, tobacco or alcohol use by school staff to 'no intervention' , 'usual' practice or a different implementation strategy.",113,113,113,113,113,113
"To assess the beneficial and harmful effects of psychological therapies for people with BPD. \n Selection Criteria: Randomised controlled trials comparing different psychotherapeutic interventions with treatment as usual TAU; which included various kinds of psychotherapy, waiting list, no treatment or active treatments in samples of all ages, in any setting, with a formal diagnosis of BPD. The primary outcomes were BPD symptom severity, self harm, suicide related outcomes, and psychosocial functioning. There were 11 secondary outcomes, including individual BPD symptoms, as well as attrition and adverse effects.",208,208,208,208,208,208


In [ ]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn import metrics
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [ ]:
folds = 5
kf = KFold(n_splits=folds, shuffle=True, random_state=0)
scores = []
y_hat = []
labels = []
for _, review in filtered_reviews:
    for train_index, test_index in kf.split(review):
        cv = TfidfVectorizer(tokenizer=lambda x: x, lowercase=False)
        x_train = cv.fit_transform(review['abstract'].values[train_index])
        x_test = cv.transform(review['abstract'].values[test_index])

        model = LogisticRegression(C=10, random_state=0, max_iter=1000).fit(x_train, review['label'].values[train_index])

        scores.append(model.score(x_test, review['label'].values[test_index]))
        y_hat.append(model.predict(x_test))
        labels.append(review['label'].values[test_index])

In [ ]:
print(metrics.classification_report(np.concatenate(y_hat), np.concatenate(labels)))

              precision    recall  f1-score   support

           0       0.83      0.81      0.82       942
           1       0.78      0.79      0.78       769

    accuracy                           0.80      1711
   macro avg       0.80      0.80      0.80      1711
weighted avg       0.80      0.80      0.80      1711



In [ ]:
df_13b = pd.read_json('../SysRev13b/eval.jsonl', lines=True)

df_13b

,instruction,input,response,label
0,"Given the abstract, what is the study's Outcome?",Abstract: Objectives Verify the influence of o...,"Below is an instruction that describes a task,...",Outcome: 1. Incidence of VAP 2. Duration of ve...
1,"Given the abstract, what is the study's Compar...",Abstract: One hundred and seventy two 0. 5 3. ...,"Below is an instruction that describes a task,...",Comparison: Randomised trial conducted in a ru...
2,"Given the abstract, objectives and selection c...",Abstract: Nutrition interventions have an effe...,"Below is an instruction that describes a task,...",Excluded because This study included moderatel...
3,"Given the abstract, what is the study's Interv...",Abstract: BACKGROUND Depression is common in p...,"Below is an instruction that describes a task,...",Intervention: Stated purpose to explore the fe...
4,"Given the abstract, what is the study's Interv...",Abstract: In the light of the current controve...,"Below is an instruction that describes a task,...","Intervention: Treatment pentoxifylline, 1200 m..."
...,...,...,...,...
7446,"Given the abstract, objectives and selection c...",Abstract: Objectives To compare by prospective...,"Below is an instruction that describes a task,...",Excluded because Insufficient follow up period
7447,"Given the abstract, what is the study's Popula...","Abstract: The changes in dust loading, lead lo...","Below is an instruction that describes a task,...",Population: Eligibility criteria Children aged...
7448,"Given the abstract, objectives and selection c...","Abstract: In this study, 59 percent of the pat...","Below is an instruction that describes a task,...",Excluded because Comparison does not meet incl...
7449,"Given the abstract, objectives and selection c...",Abstract: Background Medication adherence is a...,"Below is an instruction that describes a task,...",Included


In [ ]:
inc_exc_13b = df_13b[df_13b['instruction'].str.contains('should the study be included or excluded?')]
inc_exc_13b['prediction'] = inc_exc_13b['response'].str.split('### Response:\n').str[1].str.strip().str.split('</s>').str[0].str.split().str[0]
inc_exc_13b['label_idx'] = inc_exc_13b['label'].str.split().str[0].apply(lambda x: label_to_idx[x])
inc_exc_13b

,instruction,input,response,label,prediction,label_idx
2,"Given the abstract, objectives and selection c...",Abstract: Nutrition interventions have an effe...,"Below is an instruction that describes a task,...",Excluded because This study included moderatel...,Included,0
7,"Given the abstract, objectives and selection c...",Abstract: Objective To determine whether the b...,"Below is an instruction that describes a task,...",Included,Included,1
8,"Given the abstract, objectives and selection c...",Abstract: PURPOSE To explore the effect of exp...,"Below is an instruction that describes a task,...",Excluded because Not a task shifting intervent...,Included,0
11,"Given the abstract, objectives and selection c...",Abstract: BACKGROUND The current study was con...,"Below is an instruction that describes a task,...",Included,Included,1
14,"Given the abstract, objectives and selection c...",Abstract: The current study evaluated the effe...,"Below is an instruction that describes a task,...",Excluded because Intervention did not contain ...,Included,0
...,...,...,...,...,...,...
7445,"Given the abstract, objectives and selection c...",Abstract: Early Childhood Caries ECC is a high...,"Below is an instruction that describes a task,...",Excluded because Population children in the RC...,Included,0
7446,"Given the abstract, objectives and selection c...",Abstract: Objectives To compare by prospective...,"Below is an instruction that describes a task,...",Excluded because Insufficient follow up period,Included,0
7448,"Given the abstract, objectives and selection c...","Abstract: In this study, 59 percent of the pat...","Below is an instruction that describes a task,...",Excluded because Comparison does not meet incl...,Excluded,0
7449,"Given the abstract, objectives and selection c...",Abstract: Background Medication adherence is a...,"Below is an instruction that describes a task,...",Included,Included,1


In [ ]:
inc_exc_13b_filtered = inc_exc_13b[inc_exc_13b['prediction'].isin(['Included', 'Excluded'])]
inc_exc_13b_filtered

,instruction,input,response,label,prediction,label_idx
2,"Given the abstract, objectives and selection c...",Abstract: Nutrition interventions have an effe...,"Below is an instruction that describes a task,...",Excluded because This study included moderatel...,Included,0
7,"Given the abstract, objectives and selection c...",Abstract: Objective To determine whether the b...,"Below is an instruction that describes a task,...",Included,Included,1
8,"Given the abstract, objectives and selection c...",Abstract: PURPOSE To explore the effect of exp...,"Below is an instruction that describes a task,...",Excluded because Not a task shifting intervent...,Included,0
11,"Given the abstract, objectives and selection c...",Abstract: BACKGROUND The current study was con...,"Below is an instruction that describes a task,...",Included,Included,1
14,"Given the abstract, objectives and selection c...",Abstract: The current study evaluated the effe...,"Below is an instruction that describes a task,...",Excluded because Intervention did not contain ...,Included,0
...,...,...,...,...,...,...
7445,"Given the abstract, objectives and selection c...",Abstract: Early Childhood Caries ECC is a high...,"Below is an instruction that describes a task,...",Excluded because Population children in the RC...,Included,0
7446,"Given the abstract, objectives and selection c...",Abstract: Objectives To compare by prospective...,"Below is an instruction that describes a task,...",Excluded because Insufficient follow up period,Included,0
7448,"Given the abstract, objectives and selection c...","Abstract: In this study, 59 percent of the pat...","Below is an instruction that describes a task,...",Excluded because Comparison does not meet incl...,Excluded,0
7449,"Given the abstract, objectives and selection c...",Abstract: Background Medication adherence is a...,"Below is an instruction that describes a task,...",Included,Included,1


In [ ]:
print(metrics.classification_report(inc_exc_13b_filtered['prediction'].apply(lambda x: label_to_idx[x]), inc_exc_13b_filtered['label_idx']))

              precision    recall  f1-score   support

           0       0.20      0.86      0.32       548
           1       0.96      0.51      0.66      3819

    accuracy                           0.55      4367
   macro avg       0.58      0.68      0.49      4367
weighted avg       0.87      0.55      0.62      4367



In [ ]:
df_1_3b = pd.read_json('adapter_model.jsonl', lines=True)
inc_exc_1_3b = df_1_3b[df_1_3b['instruction'].str.contains('should the study be included or excluded?')]
inc_exc_1_3b['prediction'] = inc_exc_1_3b['response'].str.split('# # # Response:').str[1].str.strip().str.split('</s>').str[0].str.split().str[0]
inc_exc_1_3b['label_idx'] = inc_exc_1_3b['label'].str.split().str[0].apply(lambda x: label_to_idx[x])
deduped = inc_exc_1_3b[~inc_exc_1_3b[['input', 'instruction', 'response']].duplicated(keep='last')]

In [ ]:
print(metrics.classification_report(deduped['prediction'].apply(lambda x: label_to_idx[x]), deduped['label_idx']))

              precision    recall  f1-score   support

           0       0.22      0.93      0.36       898
           1       0.98      0.52      0.68      6053

    accuracy                           0.57      6951
   macro avg       0.60      0.72      0.52      6951
weighted avg       0.88      0.57      0.64      6951



In [ ]:
deduped = deduped[deduped['input'].isin(filtered_df['input'])]

In [ ]:
print(metrics.classification_report(deduped['prediction'].apply(lambda x: label_to_idx[x]), deduped['label_idx']))

              precision    recall  f1-score   support

           0       0.26      0.89      0.40       271
           1       0.96      0.52      0.68      1440

    accuracy                           0.58      1711
   macro avg       0.61      0.70      0.54      1711
weighted avg       0.85      0.58      0.63      1711

